# Induction Head Analysis Notebook

This notebook provides a step-by-step walkthrough of induction head detection and analysis in GPT-2.

## What are Induction Heads?

Induction heads are attention heads that implement a simple but powerful pattern-completion algorithm:
- When they see a token that appeared earlier in the sequence
- They attend back to that earlier occurrence
- This allows them to "copy" what came after the first occurrence

For example, in the sequence `A B C D A`, an induction head would:
1. At position 5 (second `A`), attend back to position 1 (first `A`)
2. Enable the model to predict `B` as the next token (copying the pattern)

In [ ]:
# Setup and imports
import sys
sys.path.insert(0, '..')

import torch
import matplotlib.pyplot as plt
import numpy as np

from src.model import load_model, get_model_info, get_token_strs
from src.data_gen import (
    generate_repeated_sequence,
    generate_corrupted_pair,
    generate_batch,
    get_token_set,
)
from src.analysis import (
    analyze_induction_heads,
    get_top_induction_heads,
    get_attention_pattern,
)
from src.patching import run_patching_experiment
from src.viz import (
    plot_attention_pattern,
    plot_induction_scores,
    plot_induction_heatmap,
    plot_patching_result,
)

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Load the Model

We use TransformerLens to load GPT-2 small with hooks for accessing internal activations.

In [ ]:
# Load GPT-2 small
model = load_model("gpt2-small", device="cpu")
info = get_model_info(model)
print(f"Model: {info['name']}")
print(f"Layers: {info['n_layers']}, Heads per layer: {info['n_heads']}")
print(f"Model dimension: {info['d_model']}, Head dimension: {info['d_head']}")

## 2. Generate Induction Prompts

We create prompts with repeated patterns that trigger induction behavior.

In [ ]:
# Generate a single induction prompt
vocab = get_token_set("letters")
prompt = generate_repeated_sequence(vocab, prefix_length=5, seed=42)

print(f"Prompt: {prompt.text}")
print(f"Tokens: {prompt.tokens}")
print(f"First occurrence position: {prompt.first_occurrence_pos}")
print(f"Second occurrence position: {prompt.second_occurrence_pos}")
print(f"Expected next token: {prompt.expected_next}")

In [ ]:
# Check what the model actually predicts
tokens = model.to_tokens(prompt.text, prepend_bos=True)
token_strs = get_token_strs(model, prompt.text)

print("Tokenized sequence:")
for i, t in enumerate(token_strs):
    print(f"  Position {i}: '{t}'")

# Get model prediction
with torch.no_grad():
    logits = model(tokens)
    probs = torch.softmax(logits[0, -1, :], dim=-1)
    top_probs, top_indices = torch.topk(probs, 5)

print("\nTop 5 predictions for next token:")
for prob, idx in zip(top_probs, top_indices):
    token = model.tokenizer.decode(idx.item())
    print(f"  '{token}': {prob.item():.4f}")

## 3. Analyze Induction Heads

We compute induction scores for all attention heads across multiple prompts.

In [ ]:
# Get top induction heads by averaging over multiple prompts
top_heads = get_top_induction_heads(
    model,
    vocab_name="letters",
    n_prompts=10,
    prefix_length=5,
    top_k=20,
    seed=42,
)

print("Top 10 Induction Heads:")
print("-" * 30)
for i, (layer, head, score) in enumerate(top_heads[:10]):
    print(f"{i+1}. Layer {layer}, Head {head}: {score:.4f}")

In [ ]:
# Visualize induction scores
from src.analysis import HeadScore

head_scores = [
    HeadScore(layer=l, head=h, score=s, attention_to_prev=0)
    for l, h, s in top_heads
]

fig = plot_induction_scores(head_scores, top_k=15)
plt.show()

In [ ]:
# Heatmap view
fig = plot_induction_heatmap(
    head_scores,
    n_layers=info['n_layers'],
    n_heads=info['n_heads'],
)
plt.show()

## 4. Visualize Attention Patterns

Let's look at the attention patterns of the top induction heads.

In [ ]:
# Get the top induction head
top_layer, top_head, top_score = top_heads[0]
print(f"Analyzing top induction head: Layer {top_layer}, Head {top_head} (score: {top_score:.4f})")

# Get attention pattern
attn_pattern, tokens = get_attention_pattern(model, prompt.text, top_layer, top_head)

# Plot
fig = plot_attention_pattern(
    attn_pattern,
    tokens,
    top_layer,
    top_head,
    highlight_positions=(prompt.second_occurrence_pos, prompt.first_occurrence_pos),
)
plt.show()

In [ ]:
# Compare with a non-induction head (low score)
low_layer, low_head, low_score = top_heads[-1]
print(f"Analyzing low-scoring head: Layer {low_layer}, Head {low_head} (score: {low_score:.4f})")

attn_pattern_low, _ = get_attention_pattern(model, prompt.text, low_layer, low_head)

fig = plot_attention_pattern(
    attn_pattern_low,
    tokens,
    low_layer,
    low_head,
)
plt.show()

## 5. Activation Patching Experiment

We demonstrate causal importance by patching activations from a clean run into a corrupted run.

In [ ]:
# Generate clean and corrupted prompts
clean_prompt, corrupted_prompt = generate_corrupted_pair(vocab, prefix_length=5, seed=42)

print("Clean prompt:", clean_prompt.text)
print("Corrupted prompt:", corrupted_prompt.text)
print(f"\nExpected next token: '{clean_prompt.expected_next}'")

In [ ]:
# Run patching experiment on top induction head
result = run_patching_experiment(
    model,
    clean_prompt,
    corrupted_prompt,
    patch_type="attention_head",
    layer=top_layer,
    head=top_head,
)

print(f"Patching Layer {top_layer}, Head {top_head}:")
print(f"  Clean probability: {result.clean_prob:.4f}")
print(f"  Corrupted probability: {result.corrupted_prob:.4f}")
print(f"  Patched probability: {result.patched_prob:.4f}")
print(f"  Recovery ratio: {result.recovery_ratio:.2%}")

In [ ]:
# Visualize patching result
fig = plot_patching_result(result)
plt.show()

In [ ]:
# Compare patching different heads
print("Patching results for top 5 heads:")
print("-" * 50)

results = []
for layer, head, score in top_heads[:5]:
    result = run_patching_experiment(
        model,
        clean_prompt,
        corrupted_prompt,
        patch_type="attention_head",
        layer=layer,
        head=head,
    )
    results.append((layer, head, score, result))
    print(f"L{layer}H{head} (score={score:.4f}): recovery={result.recovery_ratio:.2%}")

In [ ]:
# Residual stream patching
print("\nResidual stream patching by layer:")
print("-" * 50)

for layer in range(0, info['n_layers'], 2):  # Every other layer
    result = run_patching_experiment(
        model,
        clean_prompt,
        corrupted_prompt,
        patch_type="residual_stream",
        layer=layer,
        head=0,  # Not used for residual stream
    )
    print(f"Layer {layer}: recovery={result.recovery_ratio:.2%}")

## 6. Summary and Conclusions

### Key Findings:

1. **Induction heads are identifiable**: We can detect attention heads with high induction scores by measuring attention from repeated tokens to their first occurrence.

2. **Characteristic attention pattern**: Induction heads show a distinctive pattern of attending "back" to earlier positions with matching tokens.

3. **Causal importance**: Activation patching confirms that induction heads are causally important for pattern completion. Patching their activations from clean to corrupted runs restores the model's ability to predict the correct next token.

### In GPT-2 small:
- Induction heads typically appear in middle-to-later layers (layers 5-8)
- Not all heads are equally important; a few heads do most of the induction work
- The induction mechanism is robust across different token types and sequence lengths

In [ ]:
# Save figures for documentation
import os
os.makedirs('../figures', exist_ok=True)

# Induction scores
fig = plot_induction_scores(head_scores, top_k=15)
fig.savefig('../figures/induction_scores.png', dpi=150, bbox_inches='tight')
plt.close(fig)

# Heatmap
fig = plot_induction_heatmap(head_scores, info['n_layers'], info['n_heads'])
fig.savefig('../figures/induction_heatmap.png', dpi=150, bbox_inches='tight')
plt.close(fig)

# Attention pattern
attn_pattern, tokens = get_attention_pattern(model, prompt.text, top_layer, top_head)
fig = plot_attention_pattern(attn_pattern, tokens, top_layer, top_head)
fig.savefig('../figures/attention_pattern.png', dpi=150, bbox_inches='tight')
plt.close(fig)

print("Figures saved to ../figures/")